In [1]:
from transformers import AutoTokenizer, MBartForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained('ku-nlp/bart-base-japanese')
model = MBartForConditionalGeneration.from_pretrained('ku-nlp/bart-base-japanese')

/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
本稿では実際に計算機上で試作した論説文要約システムに関して, これで用いられている論説文要約の手法の紹介と, これによって出力された文章の評価を行う.

In [8]:
sentence = '本稿 では 実際に 計算機上 で試作した 論説文 要約システム に 関して, これで 用 いられている 論説文 要約の 手法の 紹介と, これ によって 出力された 文章の 評価を 行う.'  # input should be segmented into words by Juman++ in advance
encoding = tokenizer(sentence, return_tensors='pt')

In [9]:
encoding

{'input_ids': tensor([[    0, 18573, 24285,  1367, 28430,  3126, 28626, 28536,    12, 28884,
         28540,   198, 23644, 28673, 21607, 21699,     7,  2210,     3,   180,
         28439,   235, 21911,   630, 28441, 28433, 23644, 28673, 21607, 28428,
          3713, 28428,  1972, 28436,     3,   180,     7, 28476,    44,  3331,
         28459,   483,  7118, 28428,  1418, 28437,   794,     3,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}

In [2]:
from pyknp import Juman

jumanpp = Juman()

text = '吾輩は猫である.'
mlist = jumanpp.analysis(text)

print([mrph.midasi for mrph in mlist.mrph_list()])
# ['吾輩', 'は', '猫', 'である', '。']

Exception: Can't find JUMAN command: jumanpp

In [3]:
import nltk
nltk.download('punkt')  # 下载标点符号模型

from nltk.tokenize import sent_tokenize

japanese_text = "日本語のテキストを分割するのに使える。 英語と同様、文単位に分割することが可能。"

sentences = sent_tokenize(japanese_text)
print(sentences)


['日本語のテキストを分割するのに使える。英語と同様、文単位に分割することが可能。']


[nltk_data] Downloading package punkt to
[nltk_data]     /home/is/kaifan-l/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('ku-nlp/bart-base-japanese')
model = AutoModelForSeq2SeqLM.from_pretrained('ku-nlp/bart-base-japanese')
sentence = '京都 大学 で 自然 言語 処理 を 専攻 する 。'  # input should be segmented into words by Juman++ in advance
encoding = tokenizer(sentence, return_tensors='pt')


/home/is/kaifan-l/miniconda3/envs/summarization/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(32002, 768, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(32002, 768, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,)

In [2]:
import evaluate 

metric = evaluate.load('rouge')

In [40]:
metric.compute(predictions=['京都 大学 で を 専攻する。'], references=['京都 大学 で 自然言語 処理を専攻する。'],tokenizer=lambda x: x.split())

{'rouge1': 0.6, 'rouge2': 0.5, 'rougeL': 0.6, 'rougeLsum': 0.6}

In [34]:
from rouge import Rouge

rouge = Rouge()
predictions = ['日本語 は る 言語 である']
labels = ['日本語 は 日本 で 使われている 言語 である']

scores = rouge.get_scores(predictions, labels, avg=True)
scores


{'rouge-1': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556},
 'rouge-2': {'r': 0.3333333333333333, 'p': 0.5, 'f': 0.39999999520000007},
 'rouge-l': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556}}

In [29]:
scores


{'rouge-1': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556},
 'rouge-2': {'r': 0.3333333333333333, 'p': 0.5, 'f': 0.39999999520000007},
 'rouge-l': {'r': 0.5714285714285714, 'p': 0.8, 'f': 0.6666666618055556}}

In [21]:
from rouge import Rouge

# 参考摘要
reference_summary = "日本　摘要文本"

# 生成摘要
generated_summary = "日本　語　摘要文本"

rouge = Rouge()
scores = rouge.get_scores(generated_summary, reference_summary)

print(scores)


[{'rouge-1': {'r': 1.0, 'p': 0.6666666666666666, 'f': 0.7999999952000001}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 1.0, 'p': 0.6666666666666666, 'f': 0.7999999952000001}}]


In [47]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoConfig
config = AutoConfig.from_pretrained('ku-nlp/bart-base-japanese')
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    generation_config = AutoConfig.from_pretrained('ku-nlp/bart-base-japanese'), 
)


In [48]:
print(training_args.generation_config)

MBartConfig {
  "_name_or_path": "ku-nlp/bart-base-japanese",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "MBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
    "LABEL

In [4]:
from fugashi import GenericTagger
tagger = GenericTagger()

# parse can be used as normal
tagger.parse('something')
# features from the dictionary can be accessed by field numbers
for word in tagger(text):
    print(word.surface, word.feature[0])

RuntimeError: 
Failed initializing MeCab. Please see the README for possible solutions:

    https://github.com/polm/fugashi

If you are still having trouble, please file an issue here, and include the
ERROR DETAILS below:

    https://github.com/polm/fugashi/issues

issueを英語で書く必要はありません。

------------------- ERROR DETAILS ------------------------
arguments: [b'fugashi', b'-C']
param.cpp(69) [ifs] no such file or directory: /usr/local/etc/mecabrc
----------------------------------------------------------


In [34]:
import MeCab

def japanese_sent_tokenize(text):
    m = MeCab.Tagger('-Owakati')  # 初始化 MeCab 分词器
    sentences = []
    current_sentence = ""
    parsed = m.parse(text).split()  # 使用 MeCab 对文本进行分词
    print(parsed)
    for token in parsed:
        current_sentence += token
        if '。' in token:  # 根据句号来分割句子
            sentences.append(current_sentence)
            current_sentence = ""
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

japanese_text = "日本語のテキストを分割するのに使える。英語と同様、文単位に分割することが可能。"

japanese_sentences = japanese_sent_tokenize(japanese_text)
print(japanese_sentences)


['日本', '語', 'の', 'テキスト', 'を', '分割', 'する', 'の', 'に', '使える', '。', '英語', 'と', '同様', '、', '文', '単位', 'に', '分割', 'する', 'こと', 'が', '可能', '。']
['日本語のテキストを分割するのに使える。', '英語と同様、文単位に分割することが可能。']


In [38]:
import re
import nltk

sent_detector = nltk.RegexpTokenizer(u'[^　！？。]*[！？。.\n]')

sents = sent_detector.tokenize(u"　原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？\
      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。森田さんは「周期表に名前が残ることは感慨深い。大勢の共同研究者にまずは感謝したい」と述べた。 \n")

print(sents)
for s in sents:
    print(s)

['原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？', '      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。', '未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。', '森田さんは「周期表に名前が残ることは感慨深い。', '大勢の共同研究者にまずは感謝したい」と述べた。', ' \n']
原子番号９２のウランより重い元素は全て人工的に合成され、１１８番まで発見の報告がある？
      １１３番については、理研と米露の共同チームがそれぞれ「発見した」と報告し、国際純正・応用化学連合と国際純粋・応用物理学連合の合同作業部会が審査していた! 両学会は「データの確実性が高い」ことを理由に、理研の発見を認定し、３１日に森田さんに通知した。
未確定だった１１５番と１１７番、１１８番の新元素は米露チームの発見を認めた。
森田さんは「周期表に名前が残ることは感慨深い。
大勢の共同研究者にまずは感謝したい」と述べた。
 



In [4]:
import pandas as pd

df = pd.read_json("./datasets/tobyoki/train.json", lines=False)
df

,index,user,text,summary,incremental_summary
0,83,sachi-miu,[2014 - 10 - 29 \n 始めて みる か \n 9 月 某 日 ( て か 、...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[9 月 某 日 乳癌 告知 \n 悪性 度 高い ． 脇 と 鎖骨 の リンパ に も 転...
1,53,mamadechuyo,[2009 - 04 - 26 \n それ から ・ ・ ・ \n 4 月 20 日 に 病...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な..."
2,70,noahraisins,[2011 - 09 - 28 \n 薬 \n 今日 は 診察 日 \n\n\n ３ 週間 ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2011 - 09 - 28 診察 日 ． ホルモン 療法 3 週間 休止 中 ． 3 週...
3,45,kosoyu,"[None, 2013 - 05 - 06 \n 気合 \n 手術 から 4 日 目 、 本...","[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2013 - 04 - 11 腫瘍 内科 の 先生 の 初診 ， 乳腺 外科 の 先生 の...
4,44,kokoniiru9,[2021 - 12 - 13 \n ３ 、 入院 〜 手術 〜 \n 手術 〜 2021 ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2021 . 10 月 手術 ． \n 手術 2 日 目 ， 尿管 抜去 ． 違和感 少し...
...,...,...,...,...,...
75,57,miyukb48,[2014 - 04 - 30 \n お 父さん \n うち の 父 は 母 に ぞっこん ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な..."
76,75,omasan0108,[2017 - 05 - 01 \n びっくり さ せて しまった \n 学校 から 帰って...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[関連 イベント なし, 関連 イベント なし, 2017 - 05 - 02 ドンペリドン..."
77,32,hana333hana,[2018 - 12 - 18 \n ご 報告 ＊ \n 私 は 、 今 ある 細胞 診 検...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[2018 - 12 - 18 細胞 診 の 結果 ： 乳 がん の 診断, 2018 - ..."
78,94,usako8803,[2020 - 06 - 22 \n しこり \n 左 胸 の 下 の 部分 を 触って み...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[2020 - 06 - 19 左 胸 に しこり を みつける ．, 2020 - 06 ..."


In [1]:
import pandas as pd
data_frame = pd.read_json('./tobyoki-event_summary_juman_processed_grouped.json', orient='records', encoding='utf-8', lines=False)

In [2]:
test = data_frame.iloc[68]['text']

In [4]:
# def find_none(text):
#     for seq in text:
#         if seq == None:
#             print(seq)
#         else: 
#             print(f'{seq=}')    
#     return True
        
test

['2017 - 09 - 27 \n 今日 から ブログ を 始め ます \n 本日 2017 年 9 月 27 日 【 乳 がん 】 と 診断 さ れ ました 。 何 から 書いたら いい の か 心 の 整理 が まだ 付いて ない けど 診断 さ れたら 、 記録 と して ブログ を 始めよう と 思って いた ので 心 が 乱れて いる こと も 含めて ブログ に 、 記して おき ます 。 過去 に 、 何度 か 受けて いた 乳 がん 検診 。 いつも の ように 今年 2017 年 ２ 月 に 受診 。 でも 、 いつも と 違い 、 石灰 化 が 見つかり ました 。 念のため 、 ３ 月 に 乳腺 科 を 受診 。 「 穿刺 吸引 細胞 診 」 で 細胞 診 を した 結果 、 悪性 で は 無い と 診断 。 半年 、 様子 を 見 ましょう ！ と 言わ れ ました 。 そして 半年 後 の 受診 と して 、 9 月 12 日 に 再診 し ました 。 気 に なる シコリ が ある から と 再度 「 穿刺 吸引 細胞 診 」 し 、 次回 9 月 27 日 受診 まで 結果 を 待つ 事 に なり 、 帰り ました 。 そして 、 結果 、 悪性 細胞 が 検出 さ れた ようで 、 9 月 20 日 に 、 病院 から 電話 が あり 「 9 月 27 日 受診 の する 前 に MRI 検査 を 受けて ください 」 と 電話 が あった ので 私 と して は 、 この 時点 で 【 乳 がん 】 である 事 は 、 覚悟 して い ました 。 私 に とって ガン 宣告 さ れた の と 同様で その 日 から 、 睡眠 障害 と 、 手足 の しびれ 自律 神経 の 乱れ が 続き 自分 が どの 程度 の レベル な の か ガン に 対する 知識 が な さ 過ぎて ただただ 、 不安な 日々 が １ 週間 続き ました 。 そして 、 本日 9 月 27 日 ドクター の 口 から 【 乳 がん 】 です 。 。 。 と 診断 さ れ やっぱり そう か と 思う 気持ち の 後 、 気 に なって いた 質問 を たくさん して 少し だけ 気持ち が 落ち着いた 気 が し ます 。 そして 、 今日 は たくさ

In [38]:
# 找到shizuku-kokoro的数据
user = data_frame[data_frame['user'] == 'newgan-yuki']
user['text']

68    [2017 - 09 - 27 \n 今日 から ブログ を 始め ます \n 本日 201...
Name: text, dtype: object

In [16]:
df = data_frame[data_frame['user'] == 'shizuku-kokoro']

In [30]:
df.index

Index([85], dtype='int64')

In [23]:
for i in df['text']:
    print(i)
    print('\n')

['2016 - 06 - 19 \n こっそり 更新 状況 アップ \n メッセージ ボード に 書いて る ように 、 ブログ は 別の 場所 で やって い ます 。 これ まで ここ は 放置 でした が 、 ひっそり あっち の 更新 状況 なんて アップ して みよ と 思い つき ました 本日 更新 味覚 障害 の 時 の つよ～い 味方 関係 ない けど 、 アメブロ って 絵 文字 入れる の 簡単で いい ね', '2016 - 06 - 21 \n 更新 の お 知らせ 「 父 の 日 に 想う 」 \n こんにちは 本日 の 更新 です 父 の 日 に 想う', '2016 - 06 - 25 \n 更新 し ました 「 あれ から ７ 年 」 \n こんにちは～ 雨 降って きちゃ い ました ね 。 先 に お 買い物 行って 来れば よかった な 本日 の 更新 です あれ から ７ 年', '2016 - 06 - 27 \n 更新 し ました 「 ５ 月 6 月 の お 菓子 教室 」 \n こんにちは～ 今日 は 雨 か と 思ったら 、 意外 と お 天気 。 クリーニング 取って こよう か なぁ … 本日 の 更新 です ５ 月 ６ 月 の お 菓子 教室', '2016 - 06 - 29 \n 更新 し ました 「 外泊 に 向けて 」 \n おはよう ございます なんだか 夜中 に 目 が 覚めて 、 いろいろ ごそごそ … 早朝 から ブログ 更新 して しまった 今日 は プチ オフ で ～す が 心配だ けど 、 楽しみ 本日 の 更新 です 外泊 に 向けて', '2016 - 07 - 01 \n 更新 し ました 「 プチ オフ in 宇治 」 \n おはよう ございます 今日 から ７ 月 今年 も もう 半分 終わっちゃ った んだ 。 うそ み たい なんか 焦る わ 色々 がんばろう 本日 の 更新 です プチオフ in 宇治 ～ 紫陽花 見て 、 美味しい もの 食べよう～', '2016 - 07 - 03 \n 更新 し ました 「 美容院 へ～ 脱 ヅラ から １ 年～ 」 \n おはよう ございます めちゃめちゃ いい お 天気 梅雨 は どこ へ さて 、 今日 は １ 日 お 仕事 。

In [14]:
data_frame['user'=="shizuku-kokoro"]

KeyError: False

In [4]:
df = pd.read_json('datasets/NLP_JP_CORPUS_INCREMENTAL_JUMAN/NLP_JP_CORPUS_INCREMENTAL_JUMAN.json', orient='records', encoding='utf-8')

In [6]:
df['abs_incremental'][0]

['テキスト や 談話 を 理解 する ため に は ， まず その 文章 構造 を 理解 する 必要 が ある ．文章 構造 に 関する 従来 の 多く の 研究 で は ， 解析 に 用い られる 知識 の 問題 に 重点 が おか れて いた ．しかし ， 量的 / 質的に 十分な 計算 機 用 の 知識 が 作成 さ れる こと は しばらく の 間 期待 でき ない ．',
 'テキスト や 談話 を 理解 する ため に は ， まず その 文章 構造 を 理解 する 必要 が ある ．文章 構造 に 関する 従来 の 多く の 研究 で は ， 解析 に 用い られる 知識 の 問題 に 重点 が おか れて いた ．しかし ， 量的 / 質的に 十分な 計算 機 用 の 知識 が 作成 さ れる こと は しばらく の 間 期待 でき ない ．本 論文 で は ， 知識 に 基づく 文 理解 と いう 処理 を 行なわ ず に ， 表層 表現 中 の 種々の 情報 を 用いる こと に より 科学 技術 文 の 文章 構造 を 自動 的に 推定 する 方法 を 示す ．表層 表現 中 の 情報 と して は ， 種々の 手がかり 表現 ， 同一 / 同 義 の 語 / 句 の 出現 ， 2 文 間 の 類似 性 ， の 3 つ の もの に 着目 した ．',
 'テキスト や 談話 を 理解 する ため に は ， まず その 文章 構造 を 理解 する 必要 が ある ．文章 構造 に 関する 従来 の 多く の 研究 で は ， 解析 に 用い られる 知識 の 問題 に 重点 が おか れて いた ．しかし ， 量的 / 質的に 十分な 計算 機 用 の 知識 が 作成 さ れる こと は しばらく の 間 期待 でき ない ．本 論文 で は ， 知識 に 基づく 文 理解 と いう 処理 を 行なわ ず に ， 表層 表現 中 の 種々の 情報 を 用いる こと に より 科学 技術 文 の 文章 構造 を 自動 的に 推定 する 方法 を 示す ．表層 表現 中 の 情報 と して は ， 種々の 手がかり 表現 ， 同一 / 同 義 の 語 / 句 の 出現 ， 2 文 間 の 類似 性 ， の 3 つ の もの に 着目 した ．実

In [10]:
import pandas as pd

data_frame = pd.read_json('datasets/tobyoki/tobyoki-event_summary_juman_processed_grouped.json', orient='records', encoding='utf-8', lines=False)
data_frame

,user,text,summary,incremental_summary
0,12gatunoaozora,[2014 - 01 - 10 \n 告知 \n 2013 年 12 月 、 手術 検査 の...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[2013 年 12 月 左 乳 がん 告知 （ ステージ 0 の 初期 ）, 2013 年..."
1,1732araiguma,[2009 - 11 - 24 \n しこり 発見 o ( ＞ ＜ ) o\n 先月 末 の...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2009 年 10 月 末 左 胸 に しこり を 触知 翌日 受診 乳 がん の 可能 ...
2,2016kei,[2016 - 02 - 26 \n 人間 ドッグ (((^^;) \n 本日 から ブログ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[201 年 の 時点 で 46 歳 \n 2016 - 01 - 28 人間ドック, 20..."
3,42307191112,[2009 - 05 - 31 \n 大 運動 会 \n 上々の お 天気 で の 運動 会...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[関連 イベント なし, 2009 - 06 - 072 日 早く 月経 が はじまる 創部..."
4,6to-sei,[2008 - 09 - 30 \n 病院 の 日 \n 久しぶりに お 義父 さん に 送...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2008 - 09 - 30 ハーセプチン 投与 しびれ あり 昨 夕 より 胃 もたれ ...
...,...,...,...,...
95,vioringo,[2022 - 09 - 08 \n 自己 紹介 \n 初め まして 。 vioringo ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2021 年 10 月 乳 がん ステージ Ⅱ b と 診断 ． \n 2021 年 12...
96,y0901r1004i0816,[2016 - 03 - 06 \n 病気 の こと \n 病気 に ついて 2015 年 ...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...",[2014 年 12 月 乳 がん 告知 ． \n 2015 年 2 月 4 日 浸潤 性 ...
97,yua5020,[2017 - 12 - 04 \n 初め まして \n 今年 の 初め に 乳 がん に ...,[2017 年 初頭 乳 がん \n 2 月 温存 手術 \n 現在 ホルモン 治療 中 \...,[2017 年 初頭 乳 がん 判明 \n 2 月 温存 手術 \n 現在 ホルモン 治療 ...
98,yuki-rabbit-hutch,[2012 - 07 - 04 \n 今日 の 気分 は 。 \n レモン 味 。 \n\n...,"[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な...","[関連 イベント なし, 関連 イベント なし, 関連 イベント なし, 関連 イベント な..."


In [11]:
def truncate_max_segments(examples):
    return examples[:4]



0     [2014 - 01 - 10 \n 告知 \n 2013 年 12 月 、 手術 検査 の...
1     [2009 - 11 - 24 \n しこり 発見 o ( ＞ ＜ ) o\n 先月 末 の...
2     [2016 - 02 - 26 \n 人間 ドッグ (((^^;) \n 本日 から ブログ...
3     [2009 - 05 - 31 \n 大 運動 会 \n 上々の お 天気 で の 運動 会...
4     [2008 - 09 - 30 \n 病院 の 日 \n 久しぶりに お 義父 さん に 送...
                            ...                        
95    [2022 - 09 - 08 \n 自己 紹介 \n 初め まして 。 vioringo ...
96    [2016 - 03 - 06 \n 病気 の こと \n 病気 に ついて 2015 年 ...
97    [2017 - 12 - 04 \n 初め まして \n 今年 の 初め に 乳 がん に ...
98    [2012 - 07 - 04 \n 今日 の 気分 は 。 \n レモン 味 。 \n\n...
99    [2008 - 02 - 23 \n 強風 \n 散歩 に 行こう と 思った けど 、 寒...
Name: text, Length: 100, dtype: object

In [12]:
test[0]

['2014 - 01 - 10 \n 告知 \n 2013 年 12 月 、 手術 検査 の 結果 を 聞き に 病院 へ 。 「 残念です が 、 乳 がん です 。 」 本当に 「 残念です が ・ ・ ・ 」 って 言う んだ～ が 最初の 感想 。 「 ステージ ０ の 初期 です が 、 治療 と して は 手術 で 左 乳房 を 全摘 し ます 。 温存 は 出来 ませ ん 。 」 ・ ・ ・ 。 おっぱい 、 とっちゃ う の ？ ？ ？ ここ から 先 は 、 ほとんど 覚えて ませ ん 。 きっつい なぁ～ こんな 思い しか 出て こ なかった ・ ・ ・',
 '2014 - 01 - 14 \n 手術 は イヤ ！ \n 告知 から 数 日 、 頭 の 中 は 病気 の こと ばかり 。 気 が つけば 、 涙 も 出て くる 。 乳房 再建 に ついて インターネット で 調べて も よく わから ない 。 体験 者 の 話 は 、 イヤな 部分 だけ 目 に 付く 。 こんな 中 で 衝動 的に パパ に 話した の は 、 「 手術 は 受け ない ！ 」 でした 。 みんな 、 チビ の ため って 言う けど 、 母親 を 必要 と する の は 後 何 年間 の 話 ？ それ から 先 の 数十 年 で 、 私 は 手術 を 受けた こと を 後悔 し ない の ？ 費用 だって たくさん 掛かる なら 、 その分 チビ の 将来 に まわした ほう が ・ ・ ・ そんな モヤモヤ した 気持ち が 溜まって ・ ・ ・ パパ は 黙って 聞いて くれ ました 。 「 どう する か は 自分 で 決め なさい 。 」 私 は 本当 は どう し たい の か な ・ ・ ・',
 '2014 - 01 - 17 \n 最悪の 大晦日 \n 12 月 31 日 に 家族 で 私 の 実家 に 帰省 。 病気 が わかって から 初めて の 帰省 な ので 、 ここ で 両親 と 病気 に ついて 話し ました 。 反応 、 うすっ ！ ！ ∑(ﾟДﾟ) 「 で 、 手術 は いつ ？ 当日 行って も ヒマだ から 、 落ち着いたら お 見舞い に 行く よ 」 だって ・ ・ ・ 娘 が ガン の 手術 だって いう のに 